# Linking Education and Multistate Employment Data

### Table of Contents
1. [Introduction](#Introduction)
2. [Setting up a Database Connection](#Database-Connection)
3. [Education Queries ](#Writing-Our-Education-Database-Query)
4. [Identifying People in Multiple Datasets](#Linking-Ohio-Education-and-Employment-Microdata)
5. [Explore the Connected Data](#Exploratory-Data-Analysis)
6. [Identifying People Across Multiple States](#Linking-Ohio-Education-and-Multistate-Employment-Microdata)
7. [Additional Data Exploration:Multistate Wages](#Additional-Exploratory-Data-Analysis)
8. [Exploring Sources of Errors and Inference](#Exploring-Sources-of-Error-and-Inference)


## Introduction
- Back to [Table of Contents](#Table-of-Contents)

In this notebook, we use Ohio administrative data on earned education degrees. We will identify a specific type of education program and select a specific cohort (a group of students who graduated in the same year). 

Next, we see if we can find employment outcomes for this group of students. Here, we have to think of the type of employment outcome we are interested in. How many years after graduation and when do we identify someone as being employed? 

In addition, we have to think about where to find these employment outcomes. We can investigate the Ohio administrative data on employment, but we can also investigate if students became employed in some other states. 

By going through this notebook, you get an idea of all the choices you make while going through such a process, and you get an idea of how these decisions influences the outcomes you will find. 

We start by loading the packages we are going to use: 

In [ ]:
%pylab inline
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import time

## Database Connection
First we set our database connection parameters

- Back to [Table of Contents](#Table-of-Contents)

In [ ]:
db_name = "appliedda"
hostname = "10.10.2.10"

Next, we set our database connections, and we use the psycopg2 module so we can more easily execute queries without returning data:

In [ ]:
conn = psycopg2.connect(database=db_name, host=hostname)
cursor = conn.cursor()

## Writing Our Education Database Query 

- Back to [Table of Contents](#Table-of-Contents)

Note that with this query, we identify and select only persons who obtained a education degree with 'cip' = '23', which equates to a bachelor's degree in English. In addition, we only include persons who obtained their degree in 2010. When you are finished with going through this notebook, you can come back to this query and play around with selecting other degrees or other cohorts. 

Here we use list comprehension to shorten the list of table names in our query text

In [ ]:
level_list = ['degcert_au_inst1_level_{}'.format(i) for i in range(1, 17)] +\
['degcert_sm_inst1_level_{}'.format(i) for i in range(1, 12)] +\
['degcert_sp_inst1_level_{}'.format(i) for i in range(1, 18)] +\
['degcert_wi_inst1_level_{}'.format(i) for i in range(1, 11)]
level_cols = ','.join([l+'::text' for l in level_list])

subject_list = ['degcert_au_inst1_subject_{}'.format(i) for i in range(1, 17)] +\
['degcert_sm_inst1_subject_{}'.format(i) for i in range(1, 12)] +\
['degcert_sp_inst1_subject_{}'.format(i) for i in range(1, 18)] +\
['degcert_wi_inst1_subject_{}'.format(i) for i in range(1, 11)]
subject_cols = ','.join([s+'::text' for s in subject_list])

term_list = ['degcert_au_inst1_term_earned_{}'.format(i) for i in range(1, 17)] +\
['degcert_sm_inst1_term_earned_{}'.format(i) for i in range(1, 12)] +\
['degcert_sp_inst1_term_earned_{}'.format(i) for i in range(1, 18)] +\
['degcert_wi_inst1_term_earned_{}'.format(i) for i in range(1, 11)]
term_cols = ','.join([t+'::text' for t in term_list])

In [ ]:
# Identify:
# - (From Ohio state public institutions)
# - English (CIP 23)
# - Both 1st and 2nd bachelors
# - Multiple simultaneous English language diplomas not included 
# - This is only cohort that obtained diploma in 2010

SQL_EDU = '''
CREATE TEMP TABLE cohort_2010 AS
SELECT DISTINCT ON (key_id) key_id, format('%s-%s-1', deg_year, deg_term::int*3-2)::date yr_q
FROM (SELECT key_id, file_year AS "deg_year",
    UNNEST(array[{TERM_COLS}]) AS "deg_term",
    UNNEST(array[{SUBJECT_COLS}]) AS "cip",
    UNNEST(array[{LEVEL_COLS}]) AS "degr_lvl"
    FROM data_ohio_olda_2018.oh_hei
    WHERE file_year = '2010'
) q
WHERE deg_term IS NOT NULL
AND LEFT(cip, 2)='23'
AND (degr_lvl ='5' OR 
      degr_lvl ='22')
ORDER BY key_id, deg_term, deg_year;
COMMIT;
'''.format(TERM_COLS = term_cols, SUBJECT_COLS = subject_cols, LEVEL_COLS = level_cols)

cursor.execute(SQL_EDU)

We just generated a SQL query to select our group of interest and stored it in a TEMP TABLE. We now load it into a panda dataframe so that we are able to investigate it in a bit more detail. 

In [ ]:
# Read in the 2010 cohort 
SQL_LOAD_EDU = '''
SELECT *
FROM cohort_2010
'''
df_edu = pd.read_sql(SQL_LOAD_EDU, conn)

How many people (or actually 'degrees') are in our cohort? And how many variables

In [ ]:
df_edu.shape

What do the variables mean? 

In [ ]:
df_edu.head()

Are there persons in here who obtained multiple degrees? 

In [ ]:
df_edu['key_id'].nunique()

## Linking Ohio Education and Employment Microdata

- Back to [Table of Contents](#Table-of-Contents)

As you saw in the 'education dataset'  that we just generated, persons are identified with 'key_id'. As this is an Ohio specific identification method, we also find the corresponding SSN numbers. Apparently these are hashed, so we link the corresponding values for 'ssn_hash'. The 'ssn_hash' values corresponding to 'key_id' are found in ' data_ohio_olda_2018.oh_person. We save this in a TEMP TABLE so that we can use it later in other SQL queries.

In [ ]:
SQL_EDU_SSN_2 = """
CREATE TEMP TABLE cohort_2010_EDU_SSN AS
SELECT a.key_id, a.yr_q, b.ssn_hash
FROM cohort_2010 a
LEFT JOIN data_ohio_olda_2018.oh_person b
ON a.key_id = b.key_id;

COMMIT;
"""
cursor.execute(SQL_EDU_SSN_2)

To check if the linking worked, we also load it into a pandas dataframe:

In [ ]:
SQL_EDU_SSN_test = """
SELECT a.key_id, a.yr_q, b.ssn_hash
FROM cohort_2010 a
LEFT JOIN data_ohio_olda_2018.oh_person b
ON a.key_id = b.key_id;
"""
df_edu_ssn = pd.read_sql(SQL_EDU_SSN_test, conn)
df_edu_ssn['ssn_hash'].nunique()

Now that we have identified a cohort of students of interest, and linked their SSN numbers, we can start by looking for the employment outcomes for these students. 

We will use a very specific definition for employment here, namely 'full quarter employment one year after graduation'. Full quarter employment means that a person is employed for at least one full quarter. In order to meet this requirement, there should also an employment registration for this person at the same employed in the previous and subsequent quarter. 

As our cohort consists of people who graduated in quarters 1, 2, 3 and 4 in 2010, we are interested in employment registrations in quarters 1, 2, 3 and 4 in 2011, but to identify the 'full' quarters, we also need employment registrations of 2010 quarter 4 and 2012 quarter 1.

We start by identifying all quarters within this period from the administrative data from Ohio:

In [ ]:
SQL_cohort_OH = '''
CREATE TEMP TABLE cohort_OH AS
SELECT *, 
format('%s-%s-1', year, quarter*3-2)::date job_yrq
FROM data_ohio_olda_2018.oh_ui_wage_by_employer a
WHERE (year = 2011 OR (year = 2010 AND quarter = 4) 
        OR (year = 2012 AND quarter = 1))
    AND key_id IN (SELECT key_id FROM cohort_2010_EDU_SSN);

COMMIT;
'''
cursor.execute(SQL_cohort_OH)

Next, for all the selected quarters, we identify which meet the 'full quarter' requirement:

In [ ]:
SQL_EMP_2_OH = """
CREATE TEMP TABLE cohort_OH_full AS 
SELECT a.* 
from cohort_OH a, cohort_OH b, cohort_OH c
where a.key_id=b.key_id
    AND a.key_id=c.key_id 
    AND a.employer = b.employer 
    AND a.employer = c.employer
    AND a.job_yrq = (b.job_yrq - '3 month'::interval)::date
    AND a.job_yrq = (c.job_yrq + '3 month'::interval)::date;
    
commit;

"""
cursor.execute(SQL_EMP_2_OH)

As this 'Ohio employment' dataset we just generated also doesn't contain SSN's, we again link them:

In [ ]:
SQL_EMP_3_OH = """
CREATE TEMP TABLE cohort_OH_full_SSN AS
SELECT a.*, b.ssn_hash
    FROM cohort_OH_full a
LEFT JOIN data_ohio_olda_2018.oh_person b
    ON a.key_id = b.key_id;

COMMIT;
"""
cursor.execute(SQL_EMP_3_OH)

This 'employment dataset' we generated only contains employment information from Ohio. We are going to look at other states for employment information as well, and in the final dataset we would like to have a variable identifying from which state the employment information originated (so in which state this person is actually employed). Therefore, we now generate a variable 'state' which all has the state code '39'  for the Ohio people:

In [ ]:
SQL_EMP_4_OH = """
ALTER TABLE cohort_OH_full_SSN
ADD state text DEFAULT '39';

commit;
"""
cursor.execute(SQL_EMP_4_OH)

## Exploratory Data Analysis

- Back to [Table of Contents](#Table-of-Contents)

We now investigate how the dataset we just generated looks like:

In [ ]:
SQL_OH = '''
SELECT *
FROM cohort_OH_full_SSN
'''
df_OH = pd.read_sql(SQL_OH, conn)
df_OH.head()

In [ ]:
print('the number of individual graduates represented in the Ohio dataset is'), df_OH['key_id'].nunique()

In [ ]:
df_OH.shape

## Linking Ohio Education and Multistate Employment Microdata

- Back to [Table of Contents](#Table-of-Contents)

So apparently this dataset contains 2938 quarters with employment information from 918 individuals. Note that at this point, we did not merge the education information to the employment information yet. So this are just all the quarters meeting our definition from the people in the cohort. 

Next, we are going to see if we can also identify persons of this cohort in employment registrations in other states. We start by Missouri. Here, we use the SSN's of the Ohio education cohort to identify persons in the Missouri employment dataset, and we use the exact same definition of 'full quarter employment'.

In [ ]:
SQL_cohort_MO = '''
CREATE TEMP TABLE cohort_MO AS
SELECT *, 
format('%s-%s-1', year, quarter*3-2)::date job_yrq
FROM kcmo_lehd.mo_wage a
WHERE (year = 2011 OR (year = 2010 AND quarter = 4) 
        OR (year = 2012 AND quarter = 1))
    AND SSN IN (SELECT ssn_hash FROM cohort_2010_EDU_SSN);

COMMIT;
'''
cursor.execute(SQL_cohort_MO)

Also for Missouri, we identify which of the quarters meet the 'full quarter' requirement.

In [ ]:
SQL_cohort_MO_full = """
CREATE TEMP TABLE cohort_MO_full AS 
SELECT a.* 
from cohort_MO a, cohort_MO b, cohort_MO c
where a.ssn=b.ssn 
    AND a.ssn=c.ssn 
    AND a.empr_no = b.empr_no 
    AND a.empr_no = c.empr_no
    AND a.job_yrq = (b.job_yrq - '3 month'::interval)::date
    AND a.job_yrq = (c.job_yrq + '3 month'::interval)::date;
    
commit;

"""
cursor.execute(SQL_cohort_MO_full)

And we check how the Missouri dataset we just generated looks like. As you can see here, SSN is already in here as well as a 'state' variable, indicating this data originates from Missouri.

In [ ]:
# Check the Missouri set
SQL_MO = '''
SELECT *
FROM cohort_MO_full
'''
df_MO = pd.read_sql(SQL_MO, conn)
df_MO.head()

We now apply the same steps on data from the state Illinois: 

In [ ]:
SQL_cohort_IL_full = '''
CREATE TEMP TABLE cohort_IL AS
SELECT *, 
format('%s-%s-1', year, quarter*3-2)::date job_yrq
FROM il_des_kcmo.il_wage a
WHERE (year = 2011 OR (year = 2010 AND quarter = 4) 
        OR (year = 2012 AND quarter = 1))
    AND SSN IN (SELECT ssn_hash FROM cohort_2010_EDU_SSN);

COMMIT;
'''
cursor.execute(SQL_cohort_IL_full)

In [ ]:
SQL_EMP_2_IL = """
CREATE TEMP TABLE cohort_IL_full AS 
SELECT a.* 
from cohort_IL a, cohort_IL b, cohort_IL c
where a.ssn=b.ssn 
    AND a.ssn=c.ssn 
    AND a.empr_no = b.empr_no 
    AND a.empr_no = c.empr_no
    AND a.job_yrq = (b.job_yrq - '3 month'::interval)::date
    AND a.job_yrq = (c.job_yrq + '3 month'::interval)::date;
    
commit;

"""
cursor.execute(SQL_EMP_2_IL)

In [ ]:
SQL_IL = '''
SELECT *
FROM cohort_IL_full
'''
df_IL= pd.read_sql(SQL_IL, conn)
df_IL.head()

And the state of Indiana...

In [ ]:
SQL_cohort_IN_full = '''
CREATE TEMP TABLE cohort_IN AS
SELECT *, 
format('%s-%s-1', year, quarter*3-2)::date job_yrq
FROM in_data_2019.wages_by_employer a
WHERE (year = 2011 OR (year = 2010 AND quarter = 4) 
        OR (year = 2012 AND quarter = 1))
    AND SSN IN (SELECT ssn_hash FROM cohort_2010_EDU_SSN);

COMMIT;
'''
cursor.execute(SQL_cohort_IN_full)

In [ ]:
SQL_EMP_2_IN = """
CREATE TEMP TABLE cohort_IN_full AS 
SELECT a.* 
from cohort_IN a, cohort_IN b, cohort_IN c
where a.ssn=b.ssn 
    AND a.ssn=c.ssn 
    AND a.fein = b.fein 
    AND a.fein = c.fein
    AND a.job_yrq = (b.job_yrq - '3 month'::interval)::date
    AND a.job_yrq = (c.job_yrq + '3 month'::interval)::date;
    
commit;

"""
cursor.execute(SQL_EMP_2_IN)

In [ ]:
SQL_IN = '''
SELECT *
FROM cohort_IN_full
'''
df_IN= pd.read_sql(SQL_IN, conn)
df_IN.head()

In [ ]:
SQL_EMP_4_IN = """
ALTER TABLE cohort_IN_full
ADD state text DEFAULT '18';

commit;
"""
cursor.execute(SQL_EMP_4_IN)

We can now merge the employment information from the four different states. Note that we cannot just merge the four complete datasets as they are, as they all have some unique variables. We first identify the ones that are similar, which are: 'ssn', 'state', 'wage' and 'job_yrq'. Note that the variables are sometimes spelled a bit differently over different states. Also note that we are not including any information identifying multiple employers, as this information is not generalizable over different states. This is information we are losing by doing this merge. 

In [ ]:
SQL_MERGE_MO_IL = """
CREATE TEMP TABLE cohort_merge AS
SELECT ssn, state, wage, job_yrq FROM cohort_MO_full
UNION ALL
SELECT ssn, state, wage, job_yrq FROM cohort_IL_full
UNION ALL 
SELECT ssn, state, wages, job_yrq FROM cohort_IN_full
UNION ALL
SELECT ssn_hash, state, wages, job_yrq FROM cohort_OH_full_SSN;

commit;

"""
cursor.execute(SQL_MERGE_MO_IL)

## Additional Exploratory Data Analysis

- Back to [Table of Contents](#Table-of-Contents)

And we investigate how the merged dataset looks like: 

In [ ]:
SQL_OH_TEST = """
SELECT * FROM cohort_merge;
"""
df_test = pd.read_sql(SQL_OH_TEST, conn)
df_test['ssn'].nunique()

In [ ]:
df_test.head()

Now that we obtained all employment information for our cohort that we were able to find in the available administrative registries, we can link this information to the education cohort dataset. We use 'LEFT JOIN' in such a way that we only link the employment information exactly one year later (same quarter) to the education cohort data.

In [ ]:
SQL_link_edu_emp = """
CREATE TEMP TABLE cohort_2010_test AS
SELECT cohort_2010_EDU_SSN.*, cohort_merge.* 
FROM cohort_2010_EDU_SSN 
LEFT JOIN cohort_merge
ON cohort_2010_EDU_SSN.ssn_hash = cohort_merge.ssn
    AND cohort_2010_EDU_SSN.yr_q = (cohort_merge.job_yrq -'1 year'::interval)::date;

commit;
"""
cursor.execute(SQL_link_edu_emp)

And we again investigate this dataset

In [ ]:
SQL_OH_TEST = """
SELECT * FROM cohort_2010_test;
"""
df_test = pd.read_sql(SQL_OH_TEST, conn)

In [ ]:
df_test.head()

In [ ]:
df_test['key_id'].nunique()

In [ ]:
df_test['ssn'].nunique()

Apparently, this dataset has 1602 registrations for 1551 persons. So a couple of them had wages from multiple employers (resulting in multiple rows in this dataset). In addition, there are 686 ssns in there. SSN was only registered in the employment dataset, as we had 'key_id'  and 'ssn_hash'  in the education data. So 696 persons had employment registrations in this dataset. 

As we saw that some persons had multiple employers from whom they obtained income, there are different ways we can handle this. We can just sum all the wages for every person, or we identify the highest wage per person, and identify this as primary income. We name them 'sumwage' and 'maxwage'. Note that in the way we specified the code here, we do not sum or max wages if a person is earning wages in multiple states simultaneously. 

In [ ]:
SQL_SUM_MAX = """
CREATE TEMP TABLE cohort_2010_sumwages AS
SELECT key_id, state, 
    SUM(wage) as sumwage,
    MAX(wage) as maxwage
FROM cohort_2010_test
GROUP BY key_id, state;
commit;

"""
cursor.execute(SQL_SUM_MAX)

We can also link a dataset with auxiliary information per person:

In [ ]:
SQL_LINK_AUX = '''
select *
from cohort_2010_sumwages a
left join data_ohio_olda_2018.oh_hei_demo b
on a.key_id = b.key_id
'''
df_aux = pd.read_sql(SQL_LINK_AUX, conn)

Now we can generate variables measuring wages one year after employment. We generate a few different definitions here: either 'sum'  or 'max' of the wages, and we set different thresholds. A threshold here can be used to identify persons who for example earn less than the minimum wage with fulltime employment, or half of that value.

In [ ]:
# Generate wage variables with different definitions for employment
df_aux['max_0_singleS_fullQ']    = np.where(df_aux['maxwage'] > 0, df_aux['maxwage'], np.nan)
df_aux['sum_0_singleS_fullQ']    = np.where(df_aux['sumwage'] > 0, df_aux['sumwage'], np.nan)
df_aux['max_1924_singleS_fullQ'] = np.where(df_aux['maxwage'] > 1924, df_aux['maxwage'], np.nan)
df_aux['sum_1924_singleS_fullQ'] = np.where(df_aux['sumwage'] > 1924, df_aux['maxwage'], np.nan)
df_aux['max_3848_singleS_fullQ'] = np.where(df_aux['maxwage'] > 3848, df_aux['maxwage'], np.nan)
df_aux['sum_3848_singleS_fullQ'] = np.where(df_aux['sumwage'] > 3848, df_aux['maxwage'], np.nan)


Next, we can generate boxplots of the wages we found using the different definitions that we just generated.

In [ ]:
f, ax = plt.subplots(figsize=(16,6))
df_aux[['max_0_singleS_fullQ','sum_0_singleS_fullQ','max_1924_singleS_fullQ',
                   'sum_1924_singleS_fullQ','max_3848_singleS_fullQ','sum_3848_singleS_fullQ']].boxplot()
ax.set_ylim(0,25000);

Here, we see for example that the spread is smaller if we define a threshold and also the median is higher in such cases. Also, we see that the differences between 'sum' and 'max' are minor.

In [ ]:
df_aux[['max_0_singleS_fullQ','sum_0_singleS_fullQ','max_1924_singleS_fullQ',
                   'sum_1924_singleS_fullQ','max_3848_singleS_fullQ','sum_3848_singleS_fullQ']].count()

If we look at the number of observations per variable, we also see differences. If we would define employment using the first listed definition, the employment rate for this cohort of students English would be much higher compared to if we would have used the last definition listed.

## Exploring Sources of Error and Inference

- Back to [Table of Contents](#Table-of-Contents)

Now you can ask yourself a few questions: 

Do you think these employment rates and median wages could be used for policy purposes to evaluate the Bachelor English in Ohio? Why/why not? 

What information are we missing that might make these results less usefull? 
Information we might be missing:
- Employment of persons in other states than the ones we were able to investigate
- Employment at federal government is not included in the dataset
- People who are self-employed 

What do you think of the definitions that we made? How would you interpret the median wage values as we have no information whether the persons are full-time or part-time employed. Is setting a threshold a solution for this? 

How would the results be different for different education types and/or different cohorts? You can try this out by changing the first set of code that identifies the cohort.

How would the results be different if we used a different definition for employment. So instead of 'full quarter employment', we for example used 'single quarter employment'? You can also try this yourself. 
